In [1]:
from os import path
import os
import numpy as np
import mne
import matplotlib.pyplot
from mne.preprocessing import ICA
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne.datasets import sample
import pickle
from mne.connectivity import spectral_connectivity
from mne.viz import plot_sensors_connectivity
from mne.viz import ClickableImage  # noqa
from mne.viz import (plot_alignment, snapshot_brain_montage,
                     set_3d_view)


In [2]:
%matplotlib qt5

path= ('IAPS crudos/Casos/Medida 2/')
path_save= ('IAPS crudos/Casos/Medida 2/00_Preprocessed/')
EEG_Files = os.listdir('IAPS crudos/Casos/Medida 2/')

#Files Loading. All files in the specified path are listed... 
for file in range(1,len(EEG_Files)):
    _subject= file
    print('ANALIZING SUBJECT %02d' %(file))
    file_exist = 0
    try:
        filename = (path+EEG_Files[_subject])
        filename_save= (path_save+EEG_Files[_subject])
        raw = mne.io.read_raw_cnt(filename)
        raw.load_data()
        file_exist = 1
        print(filename)
   
    except:
        print(filename + ' not found')

    try:
        #raw=mne.io.read_raw_cnt('IAPS crudos/Casos/Medida 1/PE_I029_IAPS.cnt',preload=True)
        raw.set_eeg_reference('average', projection=False)
        raw.set_channel_types({'VEO':'eog','HEO':'eog'}) #Setting the eog channels
        #raw.plot()
        #raw.plot()
        raw.filter(1, 60, fir_design='firwin')  #  Bandpass filter
        #raw.plot()
        ica = ICA(n_components=31, method='fastica', random_state=0, max_iter=100)
        picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False,
                               stim=False, exclude='bads')
        ica.fit(raw, picks=picks, decim=3, reject=dict(mag=4e-12, grad=4000e-13),
                verbose='warning')  # low iterations -> does not fully converge
        ica.exclude = []
        n_max_eog= 5


    #Making the eog epochs 
        eog_evoked = create_eog_epochs(raw).average()
        eog_evoked.apply_baseline(baseline=(None, -0.2))
        eog_evoked.plot_joint()

    #Exlcuding the eog independent components 
        eog_inds, eog_scores= ica.find_bads_eog(raw)
        #print(eog_inds)
        #eog_scores_concatenated= np.concatenate((eog_scores[0],eog_scores[1]),axis=None)
        #eog_inds= np.where(np.abs(eog_scores[0]) >0.2)
        #eog_inds+= np.where(np.abs(eog_scores[1]) >0.2)
        #eog_inds = np.concatenate((eog_inds[0],eog_inds[1]),axis=None)
        #print('--------------',eog_inds,'------------------')
        ica.exclude = eog_inds

    #Plot the results

        #ica.plot_scores(eog_scores, exclude=eog_inds, title='Sources Related eog Artifacts')
        #ica.plot_properties(raw, picks=eog_inds)
        #ica.plot_components(eog_inds, title='Components ', colorbar=True)
        #ica.plot_sources(raw)
        #ica.plot_sources(eog_evoked)
        raw_reconst=raw.copy()
        ica.apply(raw_reconst)
        #raw_reconst.plot()
        #matplotlib.pyplot.close('all')

    #Event Definitions
        event_id_epochs = {'10': 10, '11': 11, '12': 12}
        events_epochs = mne.events_from_annotations(raw, event_id_epochs)[0]
        events_id_epochs_new = {'Negative': 10, 'Neutral': 11, 'Positive':12}
        events_id_epochs_negative = {'Negative': 10}
        events_id_epochs_neutral = {'Neutral': 11}
        events_id_epochs_positive = {'Positive': 12}
        event_IAPS_epochs ={10: 'Negative', 11: 'Neutral', 12:'Positive'}
        # raw.plot(n_channels=8, events=events_epochs, event_IAPS=event_IAPS_epochs,  event_id=events_id_epochs_new, scalings='60e-6', title='Channel', show=True, block=True)
        event_id_all = {'3': 3, '8': 8, '9': 9, '10': 10, '11':11, '12':12, '13': 13, '14': 14, '15': 15}
        events_all = mne.events_from_annotations(raw, event_id_all)[0]
        event_id_all_new = {'3': 3, '8': 8, 'Negative': 10, 'Neutral': 11,'Positive': 12, '13': 13, '14': 14, '15': 15}
        event_IAPS_all ={3: 'gray', 8: 'gray', 9: 'gray', 10: 'red', 11: 'green', 12: 'blue', 13: 'gray', 14: 'gray', 15: 'gray'}
        # raw.plot(n_channels=8, events=events_all, event_IAPS=event_IAPS_all,  event_id=event_id_all_new, scalings='60e-6', title='Channel', show=True, block=True)

    #Confirmation
        print(filename)
        processing = 0
        #confirmation = input('Do you want to process this file?: ')
        confirmation = 'y'
        if confirmation =='y':
            processing = 1

        while processing:

            #raw.plot(events=events_all, event_color=event_IAPS_all,  event_id=event_id_all_new, scalings='60e-6', title='Channel', show=True, block=True)
        ### ICA Artifact Rejection continuation
            #matplotlib.pyplot.close()
        # Epoching
            reject = {'eeg': 120e-6}
            tmin = -0.2
            tmax = 0.8
            epochs_filt_ICA = mne.Epochs(raw_reconst, events=events_epochs ,event_id=events_id_epochs_new, tmin=tmin, tmax=tmax, baseline=(-0.2,0), reject=reject, picks=('eeg'))
            epochs_raw = mne.Epochs(raw, events=events_epochs ,event_id=events_id_epochs_new, tmin=tmin, tmax=tmax, baseline=None, picks=('eeg'))
            #epochs_filt_ICA.plot(block=True)
            #matplotlib.pyplot.close()
            # epochs.plot_image(picks=epochs.ch_names, cmap="YlGnBu_r", sigma=1.) # Plot ERP average by channels
            # epochs.plot_image(picks=epochs.ch_names[0], cmap="YlGnBu_r", sigma=1.) # Plot ERP average by channel[0]
            #epochs.plot_topo_image(title='ERF images', sigma=1., fig_facecolor='w', font_color='k') # Plot topography
            #MEL_epochs_filt_ICA_bs = epochs_filt_ICA_bs["Negative"]
            #MER_epochs_filt_ICA_bs = epochs_filt_ICA_bs["Neutral"]
            # red_epochs.plot(block=True) # Plot raw epochs
            # red_epochs.plot_image(picks=red_epochs.ch_names[7], cmap="YlGnBu_r", sigma=1.) # Plot ERP average
            # green_epochs.plot_image(picks=green_epochs.ch_names[7], cmap="YlGnBu_r", sigma=1.) # Plot ERP average
            # blue_epochs.plot_image(picks=blue_epochs.ch_names[7], cmap="YlGnBu_r", sigma=1.) # Plot ERP average
            negative_epochs_filt_ICA = epochs_filt_ICA["Negative"]
            neutral_epochs_filt_ICA = epochs_filt_ICA["Neutral"]
            positive_epochs_filt_ICA = epochs_filt_ICA["Positive"]
            negative_epochs_filt = epochs_raw["Negative"]
            neutral_epochs_filt = epochs_raw["Neutral"]
            positive_epochs_filt = epochs_raw["Positive"]
            

        ### Confirmation

            #confirmation = input('Subject correctly processed y/n.\n')
            confirmation = 'y'
            if confirmation =='y':
                print('Saving subject information')

             # Saving Epochs filtered + ICA

                outfile = epochs_filt_ICA
                pickle_out = open(filename_save[:-4]+'_epochs_filt_ICA',"wb")
                pickle.dump(outfile, pickle_out)
                pickle_out.close()
              
                
                #outfile = negative_epochs_filt_ICA
                #pickle_out = open(filename_save[:-4]+'_negative_epochs_filt_ICA',"wb")
                #pickle.dump(outfile, pickle_out)
                #pickle_out.close()

                #outfile = neutral_epochs_filt_ICA
                #pickle_out = open(filename_save[:-4]+'_neutral_epochs_filt_ICA',"wb")
                #pickle.dump(outfile, pickle_out)
                #pickle_out.close()

                #outfile = positive_epochs_filt_ICA
                #pickle_out = open(filename_save[:-4]+'_positive_epochs_filt_ICA',"wb")
                #pickle.dump(outfile, pickle_out)
                #pickle_out.close()


            # Saving Epochs filtered

                #outfile = negative_epochs_filt.get_data()
                #pickle_out = open(filename_save[:-4]+'_negative_epochs_filt',"wb")
                #pickle.dump(outfile, pickle_out)
                #pickle_out.close()

                #outfile = neutral_epochs_filt.get_data()
                #pickle_out = open(filename_save[:-4]+'_neutral_epochs_filt',"wb")
                #pickle.dump(outfile, pickle_out)
                #pickle_out.close()

                #outfile = positive_epochs_filt.get_data()
                #pickle_out = open(filename_save[:-4]+'_positive_epochs_filt',"wb")
                #pickle.dump(outfile, pickle_out)
                #pickle_out.close()
                processing=0


            else:
                print('Processing subject', filename, 'again')
        
    except:
        print('Error: ',filename)
            
       
    matplotlib.pyplot.close()




ANALIZING SUBJECT 01
Reading 0 ... 438279  =      0.000 ...   438.279 secs...


<ipython-input-2-2da772d653c4>:15: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw = mne.io.read_raw_cnt(filename)


IAPS crudos/Casos/Medida 2/PE_LE009_IAPS.cnt
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 3301 samples (3.301 sec)

EOG channel index for this subject is: [32 42]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandp

240 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
240 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Saving subject information
ANALIZING SUBJECT 03
Reading 0 ... 644319  =      0.000 ...   644.319 secs...


<ipython-input-2-2da772d653c4>:15: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw = mne.io.read_raw_cnt(filename)


...

<ipython-input-2-2da772d653c4>:15: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw = mne.io.read_raw_cnt(filename)


IAPS crudos/Casos/Medida 2/PE_LE_011_IAPS.cnt
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 3301 samples (3.301 sec)

EOG channel index for this subject is: [32 42]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal band

Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
231 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Saving subject information
ANALIZING SUBJECT 28
IAPS crudos/Casos/Medida 2/00_Preprocessed not found
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 3301 samples (3.301 sec)

EOG channel index for this sub

<ipython-input-2-2da772d653c4>:15: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw = mne.io.read_raw_cnt(filename)


IAPS crudos/Casos/Medida 2/PE_LE007_IAPS.cnt
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 3301 samples (3.301 sec)

EOG channel index for this subject is: [32 42]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandp

In [ ]:
epochs=neutral_epochs_filt_ICA
data=epochs.get_data()
sfreq=1000
con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
    epochs, method='coh', mode='fourier', sfreq=sfreq, fmin=(0., 4., 8., 12., 16., 32.),
                                                             fmax=(4., 8., 12., 16., 32., 50.),
    faverage=True, tmin=0, mt_adaptive=False, n_jobs=1)

plot_sensors_connectivity(epochs.info, con[:, :, 1])

In [ ]:
print(eog_scores)
ind= np.where(np.abs(eog_scores[0]) >0.6)
print(ind[0])

In [ ]:
print(filename[:-4])
print(epochs_filt_ICA.events[:,[2]])

In [ ]:
print(negative_epochs_filt_ICA)
epochs_data=negative_epochs_filt_ICA.get_data()
epochs_data

In [ ]:
print(eog_scores)